In [0]:
import tensorflow as tf

In [2]:
if tf.gfile.Exists('./fenwicks'):
  tf.gfile.DeleteRecursively('./fenwicks')
!git clone https://github.com/fenwickslab/fenwicks.git

Cloning into 'fenwicks'...
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 1009 (delta 0), reused 1 (delta 0), pack-reused 1004
Receiving objects: 100% (1009/1009), 2.22 MiB | 12.97 MiB/s, done.
Resolving deltas: 100% (586/586), done.


In [3]:
from IPython.display import Audio
from scipy.io import wavfile
import librosa
import fenwicks as fw
import os
import numpy as np

In [4]:
BUCKET = 'gs://gs_colab'
PROJECT = 'tutorial6'
BATCH_SIZE = 128 #@param ["128", "256", "512"] {type:"raw"}
EPOCHS = 24 #@param {type:"slider", min:0, max:100, step:1}
LEARNING_RATE = 0.001 #@param ["0.001", "0.01", "0.1"] {type:"raw"}

In [5]:
fw.colab_utils.setup_gcs()

In [6]:
data_dir, work_dir = fw.io.get_gcs_dirs(BUCKET, PROJECT)

In [7]:
data_dir_local = fw.datasets.untar_data(fw.datasets.URLs.SPEECH_CMD_001, './speech001')

1489100800/1489096277 [==============================] - 11s 0us/step


In [8]:
example_audio_fn = os.path.join(data_dir_local, 'happy/012c8314_nohash_0.wav')
Audio(example_audio_fn)

In [9]:
SAMPLE_RATE = 16000
NOISE_DIR = '_background_noise_'
data_fn = os.path.join(data_dir, 'speech001_images')

In [10]:
def gen_silence():
  NUM_SEGMENT = 400
  NUM_SYNTH = 500

  path = os.path.join(data_dir_local, NOISE_DIR)
  out_path = os.path.join(data_dir_local, 'silence/')

  fw.io.create_clean_dir(out_path)  
  files = fw.io.enum_files(path, 'wav')
  
  for filename in files:
    _, samples = wavfile.read(filename)
    for i in range(NUM_SEGMENT):
      out_name = f'segment_{i}_{os.path.basename(filename)}'
      data = (samples[i * 200: i * 200 + SAMPLE_RATE] 
              * max(0, 2 * (np.random.random() - 0.25))).astype('int16')
      if data.max() != 0:
        wavfile.write(out_path + out_name, SAMPLE_RATE, data)

  for i in range(NUM_SYNTH):
    d = fw.audio_io.gen_synth_silence(sr = SAMPLE_RATE, n_rand = 4600)
    wavfile.write(os.path.join(out_path, f'new_synthesized_{i}.wav'), SAMPLE_RATE, d)

In [11]:
gen_silence()

/usr/local/lib/python3.6/dist-packages/scipy/io/wavfile.py:273: WavFileWarning:

Chunk (non-data) not understood, skipping it.



In [12]:
x_example = fw.audio_io.read_logmelspectrogram(example_audio_fn)
x_example.shape

(40, 101)

In [13]:
paths, y, labels = fw.data.data_dir_tfrecord(data_dir_local, data_fn,
                  extractor=fw.audio_io.read_logmelspectrogram, 
                  file_ext='wav', exclude_dirs=[NOISE_DIR], shuffle=True)

n_classes = len(labels)
n_all = len(y)

In [14]:
def build_nn(c=16, kernel_size=(2,5), c_dense=256, drop_rate=0.5):
  model = fw.Sequential()
  model.add(fw.layers.ConvBlk(c, convs=2, kernel_size=kernel_size))
  model.add(fw.layers.ConvBlk(c*2, convs=2, kernel_size=kernel_size))
  model.add(fw.layers.ConvBlk(c*4, convs=2, kernel_size=kernel_size))
  model.add(fw.layers.ConvBlk(c*8, convs=2, kernel_size=kernel_size))
  model.add(fw.layers.GlobalPools())
  model.add(fw.layers.DenseBN(c_dense, drop_rate=drop_rate))
  model.add(fw.layers.DenseBN(c_dense, drop_rate=drop_rate))
  model.add(fw.layers.Classifier(n_classes))
  return model

In [15]:
n_valid = n_all // 5 // 8 * 8
n_train = n_all - n_valid

total_steps = n_train // BATCH_SIZE * EPOCHS
lr_func = fw.train.cosine_lr(init_lr=LEARNING_RATE, total_steps=total_steps)
fw.plt.plot_lr_func(lr_func, total_steps)

Instructions for updating:
Colocations handled automatically by placer.


In [16]:
opt_func = fw.train.adam_optimizer(lr_func)
model_func = fw.train.get_clf_model_func(build_nn, opt_func)

In [17]:
parser = lambda x: fw.data.tfexample_numpy_image_parser(x, 40, 101, 1)
est = fw.train.get_tpu_estimator(n_train//BATCH_SIZE, model_func, work_dir, trn_bs=BATCH_SIZE, val_bs=n_valid)

INFO:tensorflow:Using config: {'_model_dir': 'gs://gs_colab/work/tutorial6/2019-04-25-19:10:18', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.101.95.154:8470"
    }
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f52b07981d0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': 'grpc://10.101.95.154:8470', '_evaluation_master': 'grpc://10.101.95.154:8470', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=419, num_shards=

In [18]:
def train_eval_fold(val_fold):
  train_input_func = lambda params: fw.data.tfrecord_ds(data_fn, parser,
    params['batch_size'], n_folds=5, val_fold_idx = val_fold, training=True)
  valid_input_func = lambda params: fw.data.tfrecord_ds(data_fn, parser,
    params['batch_size'], n_folds=5, val_fold_idx = val_fold, training=False)

  est.train(train_input_func, steps=total_steps)
  result = est.evaluate(valid_input_func, steps=1)
  fw.io.create_clean_dir(work_dir)
  return result

In [19]:
result = []
result.append(train_eval_fold(0))

INFO:tensorflow:Querying Tensorflow master (grpc://10.101.95.154:8470) for TPU system metadata.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 3939850044887200114)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 13925191730844203494)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 7129492297031605164)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 13753533922252627401)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 12710530006434351492)
INFO:tensorflow:*** Available Device: _DeviceAttribute

In [20]:
result.append(train_eval_fold(1))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:TPU job name worker
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into gs://gs_colab/work/tutorial6/2019-04-25-19:10:18/model.ckpt.
INFO:tensorflow:Initialized dataset iterators in 0 seconds
INFO:tensorflow:Installing graceful shutdown hook.
INFO:tensorflow:Creating heartbeat manager for ['/job:worker/replica:0/task:0/device:CPU:0']
INFO:tensorflow:Configuring worker heartbeat: shutdown_mode: WAIT_FOR_COORDINATOR

INFO:tensorflow:Init TPU system
INFO:tensorflow:Initialized TPU in 10 seconds
INFO:tensorflow:Starting infeed thread controller.
INFO:tensorflow:Starting outfeed thread controller.
INFO:tensorflow:Enqueue next (419) batch(es) of data to infeed.
INFO:tensorflow:Dequeue next (419) batch(es) of data from outfeed.
INFO:tensorflow:loss =

In [21]:
result.append(train_eval_fold(2))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:TPU job name worker
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into gs://gs_colab/work/tutorial6/2019-04-25-19:10:18/model.ckpt.
INFO:tensorflow:Initialized dataset iterators in 0 seconds
INFO:tensorflow:Installing graceful shutdown hook.
INFO:tensorflow:Creating heartbeat manager for ['/job:worker/replica:0/task:0/device:CPU:0']
INFO:tensorflow:Configuring worker heartbeat: shutdown_mode: WAIT_FOR_COORDINATOR

INFO:tensorflow:Init TPU system
INFO:tensorflow:Initialized TPU in 9 seconds
INFO:tensorflow:Starting infeed thread controller.
INFO:tensorflow:Starting outfeed thread controller.
INFO:tensorflow:Enqueue next (419) batch(es) of data to infeed.
INFO:tensorflow:Dequeue next (419) batch(es) of data from outfeed.
INFO:tensorflow:loss = 

In [22]:
result.append(train_eval_fold(3))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:TPU job name worker
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into gs://gs_colab/work/tutorial6/2019-04-25-19:10:18/model.ckpt.
INFO:tensorflow:Initialized dataset iterators in 0 seconds
INFO:tensorflow:Installing graceful shutdown hook.
INFO:tensorflow:Creating heartbeat manager for ['/job:worker/replica:0/task:0/device:CPU:0']
INFO:tensorflow:Configuring worker heartbeat: shutdown_mode: WAIT_FOR_COORDINATOR

INFO:tensorflow:Init TPU system
INFO:tensorflow:Initialized TPU in 10 seconds
INFO:tensorflow:Starting infeed thread controller.
INFO:tensorflow:Starting outfeed thread controller.
INFO:tensorflow:Enqueue next (419) batch(es) of data to infeed.
INFO:tensorflow:Dequeue next (419) batch(es) of data from outfeed.
INFO:tensorflow:loss =

In [23]:
result.append(train_eval_fold(4))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:TPU job name worker
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into gs://gs_colab/work/tutorial6/2019-04-25-19:10:18/model.ckpt.
INFO:tensorflow:Initialized dataset iterators in 0 seconds
INFO:tensorflow:Installing graceful shutdown hook.
INFO:tensorflow:Creating heartbeat manager for ['/job:worker/replica:0/task:0/device:CPU:0']
INFO:tensorflow:Configuring worker heartbeat: shutdown_mode: WAIT_FOR_COORDINATOR

INFO:tensorflow:Init TPU system
INFO:tensorflow:Initialized TPU in 10 seconds
INFO:tensorflow:Starting infeed thread controller.
INFO:tensorflow:Starting outfeed thread controller.
INFO:tensorflow:Enqueue next (419) batch(es) of data to infeed.
INFO:tensorflow:Dequeue next (419) batch(es) of data from outfeed.
INFO:tensorflow:loss =

In [24]:
acc = [res['accuracy'] for res in result]
loss = [res['loss'] for res in result]

print('accuracy:', np.mean(acc), '+/-', np.std(acc))
print('loss:', np.mean(loss), '+/-', np.std(loss))

accuracy: 0.97108954 +/- 0.0010820794
loss: 0.11725376 +/- 0.02825471
